# AWS Threat Hunting with Cloudtrail Logs
---

In [1]:
import polars as pl
import gzip
import re
import glob
import orjson
import ray
import io

In [2]:
pl.Config.set_fmt_str_lengths(80)

polars.config.Config

## Load Cloudtrail logs
Docs: https://docs.aws.amazon.com/awscloudtrail/latest/userguide/cloudtrail-read-log-files.html

In [3]:
@ray.remote
def read_json(filepath: str):
    if filepath.endswith(".gz"):
        with gzip.open(filepath, "r") as f:
            bytes = f.read()        
    else:
        with open(filepath, "r") as f:
            bytes = f.read()
    # TODO: Speed up one-level only JSON parsing
    logs = orjson.loads(bytes)["Records"]
    logs = [{k: str(v) for k, v in log.items()} for log in logs]
    data = pl.from_dicts(logs)
    return data


def read_glob_json(pattern: str):
    batches = []
    for filepath in glob.iglob(pattern):
        batch = read_json.remote(filepath=filepath)
        batches.append(batch)
    logs = pl.concat(ray.get(batches), how="diagonal_relaxed")
    return logs

In [4]:
logs = read_glob_json("data/*.json.gz")
logs.head(5)

2023-11-12 15:47:17,585	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


userAgent,eventID,errorMessage,userIdentity,eventType,errorCode,sourceIPAddress,eventName,eventSource,recipientAccountId,requestParameters,awsRegion,requestID,responseElements,eventVersion,eventTime,readOnly,apiVersion,additionalEventData,sharedEventID,resources,eventCategory,managementEvent
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""2e3fab5f-5252-4bfd-a432-1f7dad6fe1f8""","""The requested configuration is currently not supported. Please check the docume…","""{'type': 'IAMUser', 'principalId': 'AIDADO2GQD0K8TEF7KW1V', 'arn': 'arn:aws:iam…","""AwsApiCall""","""Client.Unsupported""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{'instancesSet': {'items': [{'imageId': 'ami-5209e150a9ee64097', 'minCount': 1,…","""sa-east-1""","""177eb3a2-283f-4267-bcf9-e8c1e89593f""","""None""","""1.05""","""2019-08-23T13:00:28Z""",null,null,null,null,null,null,null
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""376a64b2-a3b6-4ff2-8768-38f69ebfef0a""","""The requested configuration is currently not supported. Please check the docume…","""{'type': 'IAMUser', 'principalId': 'AIDADO2GQD0K8TEF7KW1V', 'arn': 'arn:aws:iam…","""AwsApiCall""","""Client.Unsupported""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{'instancesSet': {'items': [{'imageId': 'ami-c6ae2e7d00cb43daf', 'minCount': 1,…","""ap-northeast-2""","""d80fb451-d273-4056-8557-12e9fced2102""","""None""","""1.05""","""2019-08-23T13:00:29Z""",null,null,null,null,null,null,null
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""e7f52bc3-0fc4-4ef6-a9b3-49aad6dc60c9""","""Request limit exceeded.""","""{'type': 'IAMUser', 'principalId': 'AIDA9BO36HFBHKGJAO9C1', 'arn': 'arn:aws:iam…","""AwsApiCall""","""Client.RequestLimitExceeded""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{'instancesSet': {'items': [{'imageId': 'ami-5209e150a9ee64097', 'minCount': 1,…","""sa-east-1""","""e76dc363-eeaa-4a59-8709-f7822fd128884""","""None""","""1.05""","""2019-08-23T13:00:29Z""",null,null,null,null,null,null,null
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""376a64b2-a3b6-4ff2-8768-38f69ebfef0a""","""The requested configuration is currently not supported. Please check the docume…","""{'type': 'IAMUser', 'principalId': 'AIDADO2GQD0K8TEF7KW1V', 'arn': 'arn:aws:iam…","""AwsApiCall""","""Client.Unsupported""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{'instancesSet': {'items': [{'imageId': 'ami-c6ae2e7d00cb43daf', 'minCount': 1,…","""ap-northeast-2""","""d80fb451-d273-4056-8557-12e9fced2102""","""None""","""1.05""","""2019-08-23T13:00:29Z""",null,null,null,null,null,null,null
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""e7f52bc3-0fc4-4ef6-a9b3-49aad6dc60c9""","""Request limit exceeded.""","""{'type': 'IAMUser', 'principalId': 'AIDA9BO36HFBHKGJAO9C1', 'arn': 'arn:aws:iam…","""AwsApiCall""","""Client.RequestLimitExceeded""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{'instancesSet': {'items': [{'imageId': 'ami-5209e150a9ee64097', 'minCount': 1,…","""sa-east-1""","""e76dc363-eeaa-4a59-8709-f7822fd128884""","""None""","""1.05""","""2019-08-23T13:00:29Z""",null,null,null,null,null,null,null


In [5]:
# Let's take a closer look at the userIdentity
logs.item(0, "userIdentity")

"{'type': 'IAMUser', 'principalId': 'AIDADO2GQD0K8TEF7KW1V', 'arn': 'arn:aws:iam::811596193553:user/Level6', 'accountId': '811596193553', 'accessKeyId': 'AKIA3Z2XBVUDFQ9TU4MD', 'userName': 'Level6'}"

In [6]:
logs.estimated_size("gb")

2.1179624227806926

## Log Normalization

- In a previous query, we note that `sourceIPAddress` has no nulls but `arn` does
- So in absence of `arn`, we should fill null with `sourceIPAddress` (suffixed with "source_ip:")

**Operations:**
- Reorganize columns into entity, time panel format
- Make all JSON strings JSONPath parsable: replace `'` (single quote) with `"` (double quote):
  - `userIdentity`
  - `requestParameters`
  - `responseElements`
  - `additionalEventData`
  - `resources`
- Extract `arn` from `UserIdentity`
- Convert time column into datetime
- Create integer index per entity column
- Create window column
- Create `entity_id` column with `arn` if `arn` is not null else `sourceIPAddress`

In [7]:
json_fields = [
    "userIdentity",
    "requestParameters",
    "responseElements",
    "additionalEventData",
    "resources"
]
logs_with_ids = (
    logs
    # Prepare panel format
    .with_columns(
        pl.col("eventTime").str.to_datetime(),
        pl.col(json_fields).str.replace_all("'", '"')
    )
    # Unpack identity information from userIdentity JSON
    .with_columns(
        pl.col("userIdentity").str.json_path_match("$.arn").alias("arn"),
        pl.col("userIdentity").str.json_path_match("$.accountId").alias("account_id"),
    )
    # Create non-null "entity_id"
    .with_columns(
        pl.col("arn").fill_null("source_ip:" + pl.col("sourceIPAddress")).alias("entity_id")
    ) 
)
print(logs_with_ids.select(["entity_id", "eventTime"]).describe())
logs_with_ids.head()

shape: (9, 3)
┌────────────┬─────────────────────────────────┬───────────────────────────┐
│ describe   ┆ entity_id                       ┆ eventTime                 │
│ ---        ┆ ---                             ┆ ---                       │
│ str        ┆ str                             ┆ str                       │
╞════════════╪═════════════════════════════════╪═══════════════════════════╡
│ count      ┆ 1939207                         ┆ 1939207                   │
│ null_count ┆ 0                               ┆ 0                         │
│ mean       ┆ null                            ┆ null                      │
│ std        ┆ null                            ┆ null                      │
│ min        ┆ arn:aws:iam::811596193553:root  ┆ 2017-02-12 19:57:06+00:00 │
│ 25%        ┆ null                            ┆ null                      │
│ 50%        ┆ null                            ┆ null                      │
│ 75%        ┆ null                            ┆ null         

userAgent,eventID,errorMessage,userIdentity,eventType,errorCode,sourceIPAddress,eventName,eventSource,recipientAccountId,requestParameters,awsRegion,requestID,responseElements,eventVersion,eventTime,readOnly,apiVersion,additionalEventData,sharedEventID,resources,eventCategory,managementEvent,arn,account_id,entity_id
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]",str,str,str,str,str,str,str,str,str,str
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""2e3fab5f-5252-4bfd-a432-1f7dad6fe1f8""","""The requested configuration is currently not supported. Please check the docume…","""{""type"": ""IAMUser"", ""principalId"": ""AIDADO2GQD0K8TEF7KW1V"", ""arn"": ""arn:aws:iam…","""AwsApiCall""","""Client.Unsupported""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{""instancesSet"": {""items"": [{""imageId"": ""ami-5209e150a9ee64097"", ""minCount"": 1,…","""sa-east-1""","""177eb3a2-283f-4267-bcf9-e8c1e89593f""","""None""","""1.05""",2019-08-23 13:00:28 UTC,null,null,null,null,null,null,null,"""arn:aws:iam::811596193553:user/Level6""","""811596193553""","""arn:aws:iam::811596193553:user/Level6"""
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""376a64b2-a3b6-4ff2-8768-38f69ebfef0a""","""The requested configuration is currently not supported. Please check the docume…","""{""type"": ""IAMUser"", ""principalId"": ""AIDADO2GQD0K8TEF7KW1V"", ""arn"": ""arn:aws:iam…","""AwsApiCall""","""Client.Unsupported""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{""instancesSet"": {""items"": [{""imageId"": ""ami-c6ae2e7d00cb43daf"", ""minCount"": 1,…","""ap-northeast-2""","""d80fb451-d273-4056-8557-12e9fced2102""","""None""","""1.05""",2019-08-23 13:00:29 UTC,null,null,null,null,null,null,null,"""arn:aws:iam::811596193553:user/Level6""","""811596193553""","""arn:aws:iam::811596193553:user/Level6"""
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""e7f52bc3-0fc4-4ef6-a9b3-49aad6dc60c9""","""Request limit exceeded.""","""{""type"": ""IAMUser"", ""principalId"": ""AIDA9BO36HFBHKGJAO9C1"", ""arn"": ""arn:aws:iam…","""AwsApiCall""","""Client.RequestLimitExceeded""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{""instancesSet"": {""items"": [{""imageId"": ""ami-5209e150a9ee64097"", ""minCount"": 1,…","""sa-east-1""","""e76dc363-eeaa-4a59-8709-f7822fd128884""","""None""","""1.05""",2019-08-23 13:00:29 UTC,null,null,null,null,null,null,null,"""arn:aws:iam::811596193553:user/backup""","""811596193553""","""arn:aws:iam::811596193553:user/backup"""
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""376a64b2-a3b6-4ff2-8768-38f69ebfef0a""","""The requested configuration is currently not supported. Please check the docume…","""{""type"": ""IAMUser"", ""principalId"": ""AIDADO2GQD0K8TEF7KW1V"", ""arn"": ""arn:aws:iam…","""AwsApiCall""","""Client.Unsupported""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{""instancesSet"": {""items"": [{""imageId"": ""ami-c6ae2e7d00cb43daf"", ""minCount"": 1,…","""ap-northeast-2""","""d80fb451-d273-4056-8557-12e9fced2102""","""None""","""1.05""",2019-08-23 13:00:29 UTC,null,null,null,null,null,null,null,"""arn:aws:iam::811596193553:user/Level6""","""811596193553""","""arn:aws:iam::811596193553:user/Level6"""
"""Boto3/1.9.201 Python/2.7.12 Linux/4.4.0-159-generic Botocore/1.12.201""","""e7f52bc3-0fc4-4ef6-a9b3-49aad6dc60c9""","""Request limit exceeded.""","""{""type"": ""IAMUser"", ""principalId"": ""AIDA9BO36HFBHKGJAO9C1"", ""arn"": ""arn:aws:iam…","""AwsApiCall""","""Client.RequestLimitExceeded""","""5.205.62.253""","""RunInstances""","""ec2.amazonaws.com""","""811596193553""","""{""instancesSet"": {""items"": [{""imageId"": ""ami-5209e150a9ee64097"", ""minCount"": 1,…","""sa-east-1""","""e76dc363-eeaa-4a59-8709-f7822fd128884""","""Non

### Side Quest: Investigate logs without an ARN
- Approximately 3% of logs don't have an associated ARN

In [8]:
with pl.Config(fmt_str_lengths=400):
    result = (
        logs_with_ids.filter(pl.col("arn").is_null())
        .select(pl.col("userIdentity"))
        .select(pl.col("userIdentity").str.json_extract(infer_schema_length=None))
        .get_column("userIdentity").struct.unnest()
    )
    print(len(result) / len(logs_with_ids))
    print(result)
    print(result.null_count())
    print(result.unique())
    print(result.unique().select(pl.all().n_unique()))
    print(result.select("type").unique())

0.031082292916640667
shape: (60_275, 6)
┌────────────┬──────────────────────┬─────────────┬───────────┬─────────────┬──────────┐
│ type       ┆ invokedBy            ┆ principalId ┆ accountId ┆ accessKeyId ┆ userName │
│ ---        ┆ ---                  ┆ ---         ┆ ---       ┆ ---         ┆ ---      │
│ str        ┆ str                  ┆ str         ┆ str       ┆ str         ┆ str      │
╞════════════╪══════════════════════╪═════════════╪═══════════╪═════════════╪══════════╡
│ AWSService ┆ ec2.amazonaws.com    ┆ null        ┆ null      ┆ null        ┆ null     │
│ AWSService ┆ ec2.amazonaws.com    ┆ null        ┆ null      ┆ null        ┆ null     │
│ AWSService ┆ ec2.amazonaws.com    ┆ null        ┆ null      ┆ null        ┆ null     │
│ AWSService ┆ ec2.amazonaws.com    ┆ null        ┆ null      ┆ null        ┆ null     │
│ …          ┆ …                    ┆ …           ┆ …         ┆ …           ┆ …        │
│ AWSService ┆ ec2.amazonaws.com    ┆ null        ┆ null      ┆ null  

Okay we've identified four user "types" with null ARNs: `null`, `AWSService`, `AWSAccount`, `IAMUser`. Let's further investigate.

In [9]:
for identity_type in result.select("type").unique().to_series().to_list():
    print(result.filter(pl.col("type") == identity_type).unique())

shape: (2, 6)
┌─────────┬───────────┬───────────────────────┬──────────────┬─────────────┬───────────────────────┐
│ type    ┆ invokedBy ┆ principalId           ┆ accountId    ┆ accessKeyId ┆ userName              │
│ ---     ┆ ---       ┆ ---                   ┆ ---          ┆ ---         ┆ ---                   │
│ str     ┆ str       ┆ str                   ┆ str          ┆ str         ┆ str                   │
╞═════════╪═══════════╪═══════════════════════╪══════════════╪═════════════╪═══════════════════════╡
│ IAMUser ┆ null      ┆ null                  ┆ 811596193553 ┆             ┆ HIDDEN_DUE_TO_SECURIT │
│         ┆           ┆                       ┆              ┆             ┆ Y_REASONS             │
│ IAMUser ┆ null      ┆ AIDA7ZI0RCYCPBIR0OIC3 ┆ 811596193553 ┆             ┆ piper                 │
└─────────┴───────────┴───────────────────────┴──────────────┴─────────────┴───────────────────────┘
shape: (1, 6)
┌──────┬───────────┬─────────────┬──────────────┬──────────────

### Group logs into traces / windows

In [10]:
every = "60i"
entity_col = "entity_id"
data = (
    logs_with_ids
    # Perf: Pre-sort
    .sort([entity_col, "eventTime"])
    .set_sorted([entity_col, "eventTime"])
    # Group into windows
    .with_columns(pl.col("eventTime").arg_sort().over(entity_col).cast(pl.Int64).alias("index"))
    .group_by_dynamic("index", by=entity_col, every=every)
    .agg(pl.all().exclude("index"))
    # Create window ID
    .with_columns(pl.concat_str([entity_col, pl.col("eventTime").list.first()], separator=".").alias("window_id"))
    # Reset index per window_id
    .drop("index")
    .explode(pl.all().exclude(entity_col, "window_id"))
    # Reset index to start from 1 over window groups
    .with_columns(pl.col("eventTime").arg_sort().cast(pl.Int64).over("window_id").alias("index"))
    # Sort
    .sort(["window_id", "index"])
    .set_sorted(["window_id", "index"])
    # Select relevant columns
    # Unpack identity information from userIdentity JSON
    .select([
        # Window
        "window_id",
        "index",
        # Entity
        "entity_id",
        "arn",
        "sourceIPAddress",
        "userIdentity",
        # Time
        "eventTime",
        # API request
        "awsRegion",
        "requestID",
        "requestParameters",
        "userAgent",
        # API response
        "recipientAccountId",
        "responseElements",
        # Event info
        "eventCategory",
        "eventID",
        "eventName",
        "eventSource",
        "eventType",
        "eventVersion",
        # API metadata
        "apiVersion",
        "readOnly",
        # Event metadata
        "additionalEventData",
        "managementEvent",
        "sharedEventID",
        "resources",
        # Error info
        "errorCode",
        "errorMessage",
    ])
    # TODO: Smarter query optimizations via casting
    # low cardinality (ENUM-like) columns to Categorical
)
data

window_id,index,entity_id,arn,sourceIPAddress,userIdentity,eventTime,awsRegion,requestID,requestParameters,userAgent,recipientAccountId,responseElements,eventCategory,eventID,eventName,eventSource,eventType,eventVersion,apiVersion,readOnly,additionalEventData,managementEvent,sharedEventID,resources,errorCode,errorMessage
str,i64,str,str,str,str,"datetime[μs, UTC]",str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""arn:aws:iam::811596193553:root.2017-02-12 19:57:06.000000""",0,"""arn:aws:iam::811596193553:root""","""arn:aws:iam::811596193553:root""","""255.253.125.115""","""{""type"": ""Root"", ""principalId"": ""811596193553"", ""arn"": ""arn:aws:iam::8115961935…",2017-02-12 19:57:06 UTC,"""us-east-1""","""83A6C73FE87F51FF""","""None""","""[S3Console/0.4]""","""811596193553""","""None""",null,"""3038ebd2-c98a-4c65-9b6e-e22506292313""","""ListBuckets""","""s3.amazonaws.com""","""AwsApiCall""","""1.04""",null,null,null,null,null,null,null,null
"""arn:aws:iam::811596193553:root.2017-02-12 19:57:06.000000""",1,"""arn:aws:iam::811596193553:root""","""arn:aws:iam::811596193553:root""","""255.253.125.115""","""{""type"": ""Root"", ""principalId"": ""811596193553"", ""arn"": ""arn:aws:iam::8115961935…",2017-02-12 19:59:10 UTC,"""us-east-1""","""b833be53-f15d-11e6-8abe-9409ef6d52ab""","""None""","""console.amazonaws.com""","""811596193553""","""None""",null,"""22a0d9b1-deea-4d39-827b-2af7050ed3f3""","""GetAccountPasswordPolicy""","""iam.amazonaws.com""","""AwsApiCall""","""1.02""",null,null,null,null,null,null,"""NoSuchEntityException""","""The Password Policy with domain name 811596193553 cannot be found."""
"""arn:aws:iam::811596193553:root.2017-02-12 19:57:06.000000""",2,"""arn:aws:iam::811596193553:root""","""arn:aws:iam::811596193553:root""","""255.253.125.115""","""{""type"": ""Root"", ""principalId"": ""811596193553"", ""arn"": ""arn:aws:iam::8115961935…",2017-02-12 19:59:10 UTC,"""us-east-1""","""b110697b2-f15d-11e6-8abe-9409ef6d52ab""","""None""","""console.amazonaws.com""","""811596193553""","""None""",null,"""9facf7ca-cb76-4b19-940c-3de6803f7efb""","""GetAccountSummary""","""iam.amazonaws.com""","""AwsApiCall""","""1.02""",null,null,null,null,null,null,null,null
"""arn:aws:iam::811596193553:root.2017-02-12 19:57:06.000000""",3,"""arn:aws:iam::811596193553:root""","""arn:aws:iam::811596193553:root""","""255.253.125.115""","""{""type"": ""Root"", ""principalId"": ""811596193553"", ""arn"": ""arn:aws:iam::8115961935…",2017-02-12 19:59:10 UTC,"""us-east-1""","""b8382b24-f15d-11e6-8abe-9409ef6d52ab""","""None""","""console.amazonaws.com""","""811596193553""","""None""",null,"""6596d3b4-7c98-40b1-867d-f317f1dbdc18""","""ListAccountAliases""","""iam.amazonaws.com""","""AwsApiCall""","""1.02""",null,null,null,null,null,null,null,null
"""arn:aws:iam::811596193553:root.2017-02-12 19:57:06.000000""",4,"""arn:aws:iam::811596193553:root""","""arn:aws:iam::811596193553:root""","""255.253.125.115""","""{""type"": ""Root"", ""principalId"": ""811596193553"", ""arn"": ""arn:aws:iam::8115961935…",2017-02-12 19:59:10 UTC,"""us-east-1""","""b567111c6-f15d-11e6-8abe-9409ef6d52ab""","""None""","""console.amazonaws.com""","""811596193553""","""None""",null,"""9f9d038c-e5a5-443e-83d5-4cf00941d399""","""ListMFADevices""","""iam.amazonaws.com""","""AwsApiCall""","""1.02""",null,null,null,null,null,null,null,null
"""arn:aws:iam::811596193553:root.2017-02-12 19:57:06.000000""",5,"""arn:aws:iam::811596193553:root""","""arn:aws:iam::811596193553:root""","""255.253.125.115""","""{""type"": ""Root"", ""principalId"": ""811596193553"", ""arn"": ""arn:aws:iam::8115961935…",2017-02-12 19:59:10 UTC,"""us-east-1""","""b83d3435-f15d-11e6-8abe-9409ef6d52ab""","""None""","""console.amazonaws.com""","""811596193553""","""None""",null,"""4babc3a3-77b1-44b6-9940-42363d44f5b2""","""ListAccessKeys""","""iam.amazonaws.com""","""AwsApiCall""","""1.02""",null,null,null,null,null,null,null,null
"""arn:aws:iam::811596193553:root.2017-02-12 19

In [11]:
# Visualize distribution of events from entities
data.get_column("entity_id").value_counts().sort("counts", descending=True)

entity_id,counts
str,u32
"""arn:aws:iam::811596193553:user/backup""",915834
"""arn:aws:iam::811596193553:user/Level6""",905082
"""source_ip:ec2.amazonaws.com""",44158
"""arn:aws:sts::811596193553:assumed-role/flaws/i-aa2d3b42e5c6e801a""",17208
"""arn:aws:sts::811596193553:assumed-role/SecurityMonkey/secmonkey""",12354
"""arn:aws:iam::811596193553:root""",10997
"""source_ip:config.amazonaws.com""",7473
"""source_ip:lambda.amazonaws.com""",5212
"""arn:aws:iam::811596193553:user/SecurityMokey""",4522


## Detection Rules / Queries

1. Suspicious `userAgent` (not mapped to MITRE)
2. Root-access attempts
3. Steals secrets from Secrets Manager
4. Sign-in errors via AWS Console
5. Bruteforce `AssumeRole` attempts
6. CloudTrail tampering
7. GuardDuty tampering

### Suspicious `userAgent`
API request made from a suspicious `userAgent`.

In [12]:
user_agent_greylist = [
    "blackbox",
    "blackarch",
    "kali",
    "parrot",
    "pentoo",
    "powershell",
]
query = (
    data.lazy()
    .filter(pl.col("userAgent").str.contains("|".join(user_agent_greylist)))
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str
"""arn:aws:iam::811596193553:user/Level6.2020-05-07 08:41:04.000000"""
"""arn:aws:iam::811596193553:user/backup.2018-06-24 17:49:34.000000"""
"""arn:aws:iam::811596193553:user/Level6.2020-04-06 15:42:56.000000"""
"""arn:aws:iam::811596193553:user/backup.2020-04-04 08:36:11.000000"""
"""arn:aws:iam::811596193553:user/backup.2017-03-05 11:06:30.000000"""
"""arn:aws:iam::811596193553:user/backup.2020-09-24 18:09:22.000000"""
"""arn:aws:iam::811596193553:user/Level6.2019-03-02 08:34:28.000000"""
"""arn:aws:iam::811596193553:user/backup.2019-03-06 17:11:09.000000"""
"""arn:aws:iam::811596193553:user/backup.2020-06-01 15:49:40.000000"""


### Root-access attempts
- Tactic: Priviledge Escalation (TA0004)
- Technique: Valid Accounts (T1078)

In [13]:
query = (
    data.lazy()
    .filter(
        (pl.col("eventName") == "ConsoleLogin") &
        (pl.col("userIdentity").str.contains("Root"))
    )
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str
"""arn:aws:iam::811596193553:root.2017-04-10 00:38:28.000000"""
"""arn:aws:iam::811596193553:root.2017-05-24 17:21:53.000000"""
"""arn:aws:iam::811596193553:root.2017-06-29 12:53:03.000000"""
"""arn:aws:iam::811596193553:root.2018-07-05 21:08:25.000000"""
"""arn:aws:iam::811596193553:root.2017-02-20 01:46:41.000000"""
"""arn:aws:iam::811596193553:root.2018-06-16 14:33:20.000000"""
"""arn:aws:iam::811596193553:root.2018-09-30 23:48:46.000000"""
"""arn:aws:iam::811596193553:root.2017-05-28 16:40:24.000000"""
"""arn:aws:iam::811596193553:root.2018-10-16 20:30:24.000000"""


### Sign-in errors via AWS Console
- Tactic: Credential Access (TA0006)
- Technique: Brute Force (T11100)

In [14]:
query = (
    data.lazy()
    .filter(
        (pl.col("eventSource") == "signin.amazonaws.com") &
        (pl.col("eventName") == "ConsoleLogin") &
        (pl.col("responseElements").str.to_lowercase().str.contains("fail"))
    )
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str
"""source_ip:222.230.154.255.2020-01-15 10:08:24.000000"""
"""source_ip:12.80.110.252.2018-02-26 18:45:36.000000"""
"""source_ip:231.17.3.165.2018-02-26 01:12:46.000000"""
"""source_ip:8.101.151.38.2020-01-18 07:31:34.000000"""
"""source_ip:104.255.115.244.2018-03-17 17:31:20.000000"""
"""source_ip:8.120.255.102.2017-05-17 23:23:34.000000"""


### Secrets from Secrets Manager
- Tactic: Credential Access (TA0006)
- Technique: Steal Applicatio Access Token (T1528)

In [15]:
query = (
    data.lazy()
    .filter(pl.col("eventName") == "GetSecretValue")
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str
"""arn:aws:iam::811596193553:user/backup.2020-02-03 21:30:24.000000"""


### Bruteforce `AssumeRole` attempts
- Tactic: Credential Access (TA0006)
- Tactic: Brute Force (T1110)

In [16]:
event_blacklist =  [
    "AssumeRole",
    "AssumeRoleWithSAML",
    "AssumeRoleWithWebIdentity"
]
query = (
    data.lazy()
    .filter(
        (pl.col("eventSource").str.contains("sts.amazonaws.com")) &
        (pl.col("eventName").str.contains("|".join(event_blacklist))) &
        (pl.col("errorCode").str.contains("Denied"))
    )
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str
"""arn:aws:iam::811596193553:user/backup.2018-10-12 18:31:06.000000"""
"""arn:aws:iam::811596193553:user/backup.2018-10-28 20:32:54.000000"""
"""arn:aws:iam::811596193553:user/backup.2018-10-29 04:43:17.000000"""
"""arn:aws:iam::811596193553:user/backup.2018-10-28 19:22:02.000000"""
"""arn:aws:iam::811596193553:user/backup.2018-10-28 19:26:14.000000"""
"""arn:aws:iam::811596193553:user/Level6.2019-03-07 20:33:46.000000"""
"""arn:aws:iam::811596193553:user/Level6.2019-03-07 20:33:08.000000"""
"""arn:aws:iam::811596193553:user/backup.2019-04-10 08:36:47.000000"""
"""arn:aws:iam::811596193553:user/Level6.2020-03-24 14:23:59.000000"""


### CloudTrail tampering
- Tactic: Defense Evasion (TA0005)
- Technique: Impair Defenses: Disable or Modify Tools

In [17]:
event_blacklist =  [
    "DeleteTrail",
    "StopLogging",
    "UpdateTrail"
]
query = (
    data.lazy()
    .filter(pl.col("eventName").str.contains("|".join(event_blacklist)))
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str
"""arn:aws:iam::811596193553:root.2019-08-23 15:48:47.000000"""
"""arn:aws:iam::811596193553:user/Level6.2017-08-03 07:53:22.000000"""
"""arn:aws:iam::811596193553:user/Level6.2019-06-07 10:47:41.000000"""


### GuardDuty tampering
- Tactic: Defense Evasion (TA0005)
- Technique: Impair Defenses: Disable or Modify Tools

In [18]:
event_blacklist =  [
    "DeleteDetector",
    "DeleteMembers",
    "DeletePublishingDestination",
    "DisassociateMembers",
    "DisassociateFromMasterAccount",
    "RemoveTargets",
    "StopMonitoringMembers"
]
query = (
    data.lazy()
    .filter(pl.col("eventName").str.contains("|".join(event_blacklist)))
    .select("window_id")
    .unique()
)
result = query.collect(streaming=True)
result

window_id
str


## Combine Detection Results

## Cleanup Resources

In [19]:
ray.shutdown()